# Fabric SDK Python

Before getting started, make sure the Python SDK is installed properly and the network is up and running.
For more information regarding installation process and how to set up the network please refer to [this](https://github.com/hyperledger/fabric-sdk-py/blob/master/docs/source/tutorial.md).

Run the bellow code block to import an instance of a client from the network profile already provided

In [ ]:
from hfc.fabric import Client

cli = Client(net_profile="test/fixtures/network.json")

print(cli.organizations)  # orgs in the network
print(cli.peers)  # peers in the network
print(cli.orderers)  # orderers in the network
print(cli.CAs)  # ca nodes in the network

In [ ]:
org1_admin = cli.get_user(org_name='org1.example.com', name='Admin') # User instance with the Org1 admin's certs
org2_admin = cli.get_user(org_name='org2.example.com', name='Admin') # User instance with the Org2 admin's certs
orderer_admin = cli.get_user(org_name='orderer.example.com', name='Admin') # User instance with the orderer's certs

Run the bellow code snippet to create a channel with the name 'business' channel.
This command will return True if the channel was created successfully.

In [ ]:
# Create a New Channel, the response should be true if succeed
response = await cli.channel_create(
                    orderer='orderer.example.com',
                    channel_name='businesschannel',
                    requestor=org1_admin,
                    config_yaml='test/fixtures/e2e_cli/',
                    channel_profile='TwoOrgsChannel'
                    )

# response = true is returned if the channel is created successfully
print(response)

Run the below code snippet to make the peers of Org1 join the channel.

In [ ]:
# Join Peers into Channel, the response should be true if succeed
responses = await cli.channel_join(
                       requestor=org1_admin,
                       channel_name='businesschannel',
                       peers=['peer0.org1.example.com',
                      'peer1.org1.example.com'],
                       orderer='orderer.example.com'
                       )

# The length of the response should be two as len(peers) = 2
print(len(responses) == 2)

Run the below code snippet to make the peers of Org1 join the channel.

In [ ]:
# For operations on peers from org2.example.com, org2_admin is required as requestor
responses = await cli.channel_join(
                       requestor=org2_admin,
                       channel_name='businesschannel',
                       peers=['peer0.org2.example.com',
                      'peer1.org2.example.com'],
                       orderer='orderer.example.com'
                       )

# The length of the response should be two as len(peers) = 2
print(len(responses) == 2)